In [2]:
import pandas as pd
import numpy as np

from sklearn import preprocessing

from sklearn.model_selection import train_test_split 
from sklearn.metrics import log_loss
from sklearn.tree import DecisionTreeClassifier

In [3]:
# Read all files
df_train = pd.read_csv('./train_loans2013_2015q1_minimalprocessing.csv') #download from kaggle/async 
df_test = pd.read_csv('./test_loans2013_2015q1_minimalprocessing.csv') #download from kaggle/async
print(df_train.shape, df_test.shape)  
display(df_train.head())
display(df_test.head())

/var/folders/km/lr9wgyjj0z737tmwxblc5d980000gn/T/ipykernel_2347/3877676683.py:2: DtypeWarning: Columns (48,128,129,130,133,134,135,138) have mixed types. Specify dtype option on import or set low_memory=False.
  df_train = pd.read_csv('./train_loans2013_2015q1_minimalprocessing.csv') #download from kaggle/async


(330827, 152) (82707, 150)


/var/folders/km/lr9wgyjj0z737tmwxblc5d980000gn/T/ipykernel_2347/3877676683.py:3: DtypeWarning: Columns (48,128,129,130,133,134,135,138) have mixed types. Specify dtype option on import or set low_memory=False.
  df_test = pd.read_csv('./test_loans2013_2015q1_minimalprocessing.csv') #download from kaggle/async


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term,loan_status,good_loan
0,10067489,NaN,7125,7125,7125.0,36 months,16.24%,251.34,C,C5,...,Cash,Y,Apr-2015,COMPLETE,Oct-2014,2669.0,42.17,0.0,Charged Off,1
1,40971818,NaN,10000,10000,10000.0,36 months,13.33%,338.54,C,C3,...,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN,Fully Paid,0
2,30255898,NaN,20000,20000,19900.0,36 months,11.67%,661.14,B,B4,...,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN,Charged Off,1
3,42344920,NaN,28000,28000,28000.0,36 months,12.69%,939.26,C,C2,...,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN,Fully Paid,0
4,42414207,NaN,10000,10000,9975.0,36 months,12.69%,335.45,C,C2,...,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN,Charged Off,1


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,3527341,NaN,28000,NaN,NaN,NaN,10.16%,NaN,B,B1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,40038680,NaN,20000,NaN,NaN,NaN,8.67%,NaN,B,B1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,31547538,NaN,4800,NaN,NaN,NaN,13.35%,NaN,C,C2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,38792240,NaN,21000,NaN,NaN,NaN,8.67%,NaN,B,B1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,16231995,NaN,6000,NaN,NaN,NaN,12.49%,NaN,B,B5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# Check target distribution
print(df_train.loan_status.value_counts(normalize=True),"\n\n")
print(df_train.good_loan.value_counts(normalize=True)) #Fully paid = 0 and Default = 1 

loan_status
Fully Paid     0.820299
Charged Off    0.179701
Name: proportion, dtype: float64 


good_loan
0    0.820299
1    0.179701
Name: proportion, dtype: float64


In [6]:
print(list(df_train.columns))

['id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'term', 'int_rate', 'installment', 'grade', 'sub_grade', 'emp_title', 'emp_length', 'home_ownership', 'annual_inc', 'verification_status', 'issue_d', 'pymnt_plan', 'url', 'desc', 'purpose', 'title', 'zip_code', 'addr_state', 'dti', 'delinq_2yrs', 'earliest_cr_line', 'fico_range_low', 'fico_range_high', 'inq_last_6mths', 'mths_since_last_delinq', 'mths_since_last_record', 'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc', 'initial_list_status', 'out_prncp', 'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int', 'total_rec_late_fee', 'recoveries', 'collection_recovery_fee', 'last_pymnt_d', 'last_pymnt_amnt', 'next_pymnt_d', 'last_credit_pull_d', 'last_fico_range_high', 'last_fico_range_low', 'collections_12_mths_ex_med', 'mths_since_last_major_derog', 'policy_code', 'application_type', 'annual_inc_joint', 'dti_joint', 'verification_status_joint', 'acc_now_delinq', 'tot_coll_amt'

In [7]:
#drop leakage columns from training
print("\n","df_train.shape: ", df_train.shape)
df_train.drop(['out_prncp', 'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv', 
         'total_rec_prncp','total_rec_int','total_rec_late_fee', 'recoveries','collection_recovery_fee',
         'last_pymnt_amnt',
        'debt_settlement_flag',
         'disbursement_method', 'funded_amnt', 'funded_amnt_inv', 'installment', 'issue_d', 'last_pymnt_d' , 
         'pymnt_plan' , 'term' ,  
        'last_credit_pull_d', 'last_fico_range_high', 'last_fico_range_low'
        ],axis=1,inplace=True) 

print("\n","df_train.shape: ", df_train.shape)


#drop leakage columns from test set as well
print("\n","df_test.shape: ", df_test.shape)
df_test.drop(['out_prncp', 'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv', 
         'total_rec_prncp','total_rec_int','total_rec_late_fee', 'recoveries','collection_recovery_fee',
         'last_pymnt_amnt',
        'debt_settlement_flag',
         'disbursement_method', 'funded_amnt', 'funded_amnt_inv', 'installment', 'issue_d', 'last_pymnt_d' , 
         'pymnt_plan' , 'term' ,  
        'last_credit_pull_d', 'last_fico_range_high', 'last_fico_range_low'
        ],axis=1,inplace=True) 

print("\n","df_test.shape: ", df_test.shape)


 df_train.shape:  (330827, 152)

 df_train.shape:  (330827, 130)

 df_test.shape:  (82707, 150)

 df_test.shape:  (82707, 128)


In [8]:
# Drop columns with < 25% of data available. 

lack_of_data_idx = [x for x in df_train.count() < df_train.shape[0]*0.25]
cols_to_drop = list(df_train.columns[lack_of_data_idx])

print(len(cols_to_drop))
#drop
df_train.drop(cols_to_drop, axis=1, inplace=True)
df_test.drop(cols_to_drop, axis=1, inplace=True)


print("\n(train) After dropping columns with <25% data:",df_train.shape)
print("\n(test)  After dropping columns with <25% data:",df_test.shape)




54

(train) After dropping columns with <25% data: (330827, 76)

(test)  After dropping columns with <25% data: (82707, 74)


In [9]:
len(cols_to_drop)

54

In [10]:
# Convert categorical data to numeric

df_train.columns[df_train.dtypes=='object']

Index(['int_rate', 'grade', 'sub_grade', 'emp_title', 'emp_length',
       'home_ownership', 'verification_status', 'url', 'purpose', 'title',
       'zip_code', 'addr_state', 'earliest_cr_line', 'revol_util',
       'initial_list_status', 'application_type', 'hardship_flag',
       'loan_status'],
      dtype='object')

In [11]:
df_train.emp_length.head(10)

0      3 years
1       1 year
2      3 years
3    10+ years
4      3 years
5      5 years
6    10+ years
7      8 years
8      2 years
9      5 years
Name: emp_length, dtype: object

In [12]:
df_train.verification_status.value_counts()

verification_status
Source Verified    115388
Verified           113310
Not Verified       102129
Name: count, dtype: int64

In [13]:
df_train.purpose.value_counts()

purpose
debt_consolidation    199721
credit_card            78462
home_improvement       17964
other                  14878
major_purchase          5542
small_business          3243
medical                 2939
car                     2635
moving                  1826
vacation                1660
house                   1292
wedding                  496
renewable_energy         169
Name: count, dtype: int64

In [14]:
df_train.title.nunique()

26715

In [15]:
df_train.shape, df_test.shape

((330827, 76), (82707, 74))

In [16]:
#Merge training and test set for easier preprocessing 
df_train['type_train_test'] = 'train'
df_test['type_train_test'] = 'test'

df = pd.concat([df_train, df_test])
print(df.shape)

(413534, 77)


In [17]:
#Convert purpose to dummies
purpose_cols = pd.get_dummies(df.purpose).columns
print(pd.unique(df.purpose))
df = pd.concat([df,pd.get_dummies(df.purpose)],axis=1)
df.drop('purpose',axis=1,inplace=True)

['debt_consolidation' 'credit_card' 'home_improvement' 'other' 'wedding'
 'major_purchase' 'small_business' 'car' 'moving' 'house' 'medical'
 'vacation' 'renewable_energy']


In [16]:
#Convert emp_length to numeric
df['emp_length'] = df.emp_length.map({'< 1 year':0,'1 year':1,'2 years':2 , 
                            '3 years':3,'4 years':4, '5 years':5,
                           '6 years':6, '7 years':7, '8 years':8,
                           '9 years':9, '10+ years':10})


In [17]:
#DROP EMP_TITLE
df.drop(['emp_title'], axis=1, inplace = True)


In [18]:
#Convert Verification Status
verification_status_cols = pd.get_dummies(df.verification_status).columns
df = pd.concat([df,pd.get_dummies(df.verification_status)],axis=1)
df.drop('verification_status',axis=1,inplace=True)
print(verification_status_cols)


Index(['Not Verified', 'Source Verified', 'Verified'], dtype='object')


In [19]:
#Convert categorical data
categorical = list(df.select_dtypes(include='object').columns)
#convert grade and subgrade later
categorical = ['home_ownership', 'url', 'title', 'zip_code', 'addr_state', 
               'earliest_cr_line', 'revol_util', 'initial_list_status',
               'application_type', 'hardship_flag']

print(categorical)

# Encoder (Should be done separately for train, test)
lbl = preprocessing.LabelEncoder()
for col in categorical:
    df[col] = lbl.fit_transform(df[col].astype(str))
    
df = df.fillna(-999)

['home_ownership', 'url', 'title', 'zip_code', 'addr_state', 'earliest_cr_line', 'revol_util', 'initial_list_status', 'application_type', 'hardship_flag']


In [20]:
df['int_rate'] = df['int_rate'].str.replace('%','')
df['int_rate'] = df['int_rate'].astype(np.float32)


In [21]:
categorical = ['grade', 'sub_grade']

lbl = preprocessing.LabelEncoder()
for col in categorical:
    df[col] = lbl.fit_transform(df[col].astype(str))

In [22]:
df.shape

(413534, 90)

In [23]:
df.columns

Index(['id', 'loan_amnt', 'int_rate', 'grade', 'sub_grade', 'emp_length',
       'home_ownership', 'annual_inc', 'url', 'title', 'zip_code',
       'addr_state', 'dti', 'delinq_2yrs', 'earliest_cr_line',
       'fico_range_low', 'fico_range_high', 'inq_last_6mths',
       'mths_since_last_delinq', 'open_acc', 'pub_rec', 'revol_bal',
       'revol_util', 'total_acc', 'initial_list_status',
       'collections_12_mths_ex_med', 'mths_since_last_major_derog',
       'policy_code', 'application_type', 'acc_now_delinq', 'tot_coll_amt',
       'tot_cur_bal', 'total_rev_hi_lim', 'acc_open_past_24mths',
       'avg_cur_bal', 'bc_open_to_buy', 'bc_util', 'chargeoff_within_12_mths',
       'delinq_amnt', 'mo_sin_old_il_acct', 'mo_sin_old_rev_tl_op',
       'mo_sin_rcnt_rev_tl_op', 'mo_sin_rcnt_tl', 'mort_acc',
       'mths_since_recent_bc', 'mths_since_recent_bc_dlq',
       'mths_since_recent_inq', 'mths_since_recent_revol_delinq',
       'num_accts_ever_120_pd', 'num_actv_bc_tl', 'num_actv_rev_

In [24]:
#Dropping because we have good_loan as target variable
df.drop('loan_status', axis=1,inplace=True)

In [25]:
# dividing train and test set back 
df_train = df[df.type_train_test=='train'].reset_index(drop=True)
print(df_train.shape)

df_test = df[df.type_train_test=='test'].reset_index(drop=True)
print(df_test.shape)

(330827, 89)
(82707, 89)


In [26]:
df_train.drop(['type_train_test'],axis=1,inplace=True)
df_test.drop(['type_train_test'],axis=1,inplace=True)

In [27]:
df_train.columns

Index(['id', 'loan_amnt', 'int_rate', 'grade', 'sub_grade', 'emp_length',
       'home_ownership', 'annual_inc', 'url', 'title', 'zip_code',
       'addr_state', 'dti', 'delinq_2yrs', 'earliest_cr_line',
       'fico_range_low', 'fico_range_high', 'inq_last_6mths',
       'mths_since_last_delinq', 'open_acc', 'pub_rec', 'revol_bal',
       'revol_util', 'total_acc', 'initial_list_status',
       'collections_12_mths_ex_med', 'mths_since_last_major_derog',
       'policy_code', 'application_type', 'acc_now_delinq', 'tot_coll_amt',
       'tot_cur_bal', 'total_rev_hi_lim', 'acc_open_past_24mths',
       'avg_cur_bal', 'bc_open_to_buy', 'bc_util', 'chargeoff_within_12_mths',
       'delinq_amnt', 'mo_sin_old_il_acct', 'mo_sin_old_rev_tl_op',
       'mo_sin_rcnt_rev_tl_op', 'mo_sin_rcnt_tl', 'mort_acc',
       'mths_since_recent_bc', 'mths_since_recent_bc_dlq',
       'mths_since_recent_inq', 'mths_since_recent_revol_delinq',
       'num_accts_ever_120_pd', 'num_actv_bc_tl', 'num_actv_rev_

In [28]:
df_train.to_csv('train_cleaned_jan24.csv',index=False)
df_test.to_csv('test_cleaned_jan24.csv',index=False)

In [29]:
df_train.shape, df_test.shape

((330827, 88), (82707, 88))

In [30]:
df_test.columns

Index(['id', 'loan_amnt', 'int_rate', 'grade', 'sub_grade', 'emp_length',
       'home_ownership', 'annual_inc', 'url', 'title', 'zip_code',
       'addr_state', 'dti', 'delinq_2yrs', 'earliest_cr_line',
       'fico_range_low', 'fico_range_high', 'inq_last_6mths',
       'mths_since_last_delinq', 'open_acc', 'pub_rec', 'revol_bal',
       'revol_util', 'total_acc', 'initial_list_status',
       'collections_12_mths_ex_med', 'mths_since_last_major_derog',
       'policy_code', 'application_type', 'acc_now_delinq', 'tot_coll_amt',
       'tot_cur_bal', 'total_rev_hi_lim', 'acc_open_past_24mths',
       'avg_cur_bal', 'bc_open_to_buy', 'bc_util', 'chargeoff_within_12_mths',
       'delinq_amnt', 'mo_sin_old_il_acct', 'mo_sin_old_rev_tl_op',
       'mo_sin_rcnt_rev_tl_op', 'mo_sin_rcnt_tl', 'mort_acc',
       'mths_since_recent_bc', 'mths_since_recent_bc_dlq',
       'mths_since_recent_inq', 'mths_since_recent_revol_delinq',
       'num_accts_ever_120_pd', 'num_actv_bc_tl', 'num_actv_rev_

### Create models 

In [31]:
df_train = pd.read_csv('train_cleaned_jan24.csv')

In [32]:
### Separate out features and target column

X = np.array(df_train.loc[:, df_train.columns != 'good_loan'])
y = np.array(df_train.good_loan)

In [33]:
### Creating a validation set 

#80-20 split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42,stratify=y)

print(pd.Series(y_train).value_counts(normalize=True), "\n\n")

pd.Series(y_val).value_counts(normalize=True)

0.0    0.820298
1.0    0.179702
dtype: float64 




0.0    0.8203
1.0    0.1797
dtype: float64

In [34]:
### Decision Tree

clf = DecisionTreeClassifier(max_depth=5, random_state=10) 
#max_depth by default is set to None, which leads to overfitting. max_depth is sort of regularizing parameter here.
clf.fit(X_train, y_train)
preds = clf.predict_proba(X_val)

In [35]:
print("Decision Tree:",log_loss(y_val, preds[:,1]))

Decision Tree: 0.4326988556409903
